In [250]:
from random import sample
import pprint

In [260]:
## Should work as long as num_teams * max_team_size = num_devs
num_teams = 3
num_devs = 9
max_team_size = 3

In [261]:
def setup_ranking(num_teams, num_devs):
    '''
    Creates randomized team and dev ranking list.
    
    Parameters
    -----------
    num_teams: integer
        number of teams to match
    num_devs: integer
        number of devs to match
    
    Returns
    -----------
    teams: list
        list containing names of all teams
    devs: list
        list containing names of all devs
    teams_rankings: dictionary
        key is team name, value is list of devs where lower index is higher preference
    devs_rankings: dictionary
        key is dev name, value is list of teams where lower index is higher preference
    '''
    teams = ["team" + str(team_number + 1) for team_number in range(num_teams)]
    devs = ["dev" + str(dev_number + 1) for dev_number in range(num_devs)]
    teams_rankings = dict(zip(teams, [devs] * num_teams))
    devs_rankings = dict(zip(devs, [teams] * num_devs))
    
    for k, v in teams_rankings.items():
        teams_rankings[k] = sample(v, len(v))
    for k, v in devs_rankings.items():
        devs_rankings[k] = sample(v, len(v))
        
    return teams, devs, teams_rankings, devs_rankings

def stable_marriage(teams, devs, teams_rankings, devs_rankings):
    '''
    Performs stable matching algorithm where each team can propose to up to `num_teams` number of devs
    
    Parameters
    -----------
    teams: list
        list containing names of all teams
    devs: list
        list containing names of all devs
    teams_rankings: dictionary
        key is team name, value is list of devs where lower index is higher preference
    devs_rankings: dictionary
        key is dev name, value is list of teams where lower index is higher preference
    
    Returns
    -----------
    dev_matching: dictionary
        key is dev, value is the team dev is matched to
    '''
    
    matching = [{team: []} for team in teams]
    team_availability = dict(zip(teams, ["free"] * len(teams)))
    team_size = dict(zip(teams, [0] * len(teams)))
    dev_availability = dict(zip(devs, ["free"] * len(devs)))
    team_counter = dict(zip(teams, [0] * len(teams)))
    dev_matching = {}
    
    while not matching_completed(team_availability):
        if proposals_completed(team_counter):
            break
        
        for team in teams:
            if team_availability[team] != "free" or team_counter[team] >= num_devs:
                continue

            dev_index = team_counter[team]
            dev = teams_rankings[team][dev_index]

            if dev_availability[dev] == "free":
                dev_availability[dev] = "engaged"
                dev_matching[dev] = team
                team_size[team] += 1
                if team_size[team] >= max_team_size:
                    team_availability[team] = "engaged"

            elif dev_availability[dev] == "engaged":
                matched_team = dev_matching[dev]
                proposed_team = team
                dev_rankings = devs_rankings[dev]
                if dev_rankings.index(proposed_team) < dev_rankings.index(matched_team):
                    team_size[matched_team] -= 1
                    if team_availability[matched_team] == "engaged":
                        team_availability[matched_team] = "free"

                    team_size[proposed_team] += 1
                    if team_size[proposed_team] >= max_team_size:
                        team_availability[proposed_team] = "engaged"

                    dev_matching[dev] = proposed_team

            team_counter[team] += 1
    return dev_matching

def matching_completed(team_availability):
    '''
    Returns true if all teams have been filled to capacity.
    
    Parameters
    -----------
    team_availability: dictionary
        key is team, value is string in ["free", "engaged"]

    Returns
    -----------
    boolean
    '''
    for _, availability in team_availability.items():
        if availability == "free":
            return False
    return True

def proposals_completed(team_counter):
    '''
    Returns true if all teams have proposed to all devs
    
    Parameters
    -----------
    team_counter: dictionary
        key is team, value is number of devs team has proposed to

    Returns
    -----------
    boolean
    '''
    for _, count in team_counter.items():
        if count < num_devs - 1:
            return False
    return True

def dev_to_team_matching(dev_matching):
    '''
    Converts dev-to-team matching to team-to-dev matching
    
    Parameters
    -----------
    dev_matching: dictionary
        key is dev, value is the team dev is matched to

    Returns
    -----------
    team_matching: dictionary
        key is team, value is list containing devs matched to that team
    '''
    team_matching = {}
    for dev, team in dev_matching.items():
        if team in team_matching.keys():
            team_matching[team].append(dev)
        else:
            team_matching[team] = [dev]
    return team_matching

In [262]:
teams, devs, teams_rankings, devs_rankings = setup_ranking(num_teams, num_devs)

In [263]:
dev_matching = stable_marriage(teams, devs, teams_rankings, devs_rankings)
team_matching = dev_to_team_matching(dev_matching)
pprint.pprint(team_matching)

{'team1': ['dev3', 'dev1', 'dev4'],
 'team2': ['dev2', 'dev9', 'dev8'],
 'team3': ['dev7', 'dev6', 'dev5']}


In [264]:
## Print rankings to sanity check
pprint.pprint(teams_rankings)
pprint.pprint(devs_rankings)

{'team1': ['dev2',
           'dev3',
           'dev6',
           'dev4',
           'dev9',
           'dev1',
           'dev5',
           'dev7',
           'dev8'],
 'team2': ['dev8',
           'dev7',
           'dev2',
           'dev6',
           'dev9',
           'dev5',
           'dev3',
           'dev1',
           'dev4'],
 'team3': ['dev9',
           'dev6',
           'dev7',
           'dev4',
           'dev5',
           'dev8',
           'dev3',
           'dev2',
           'dev1']}
{'dev1': ['team2', 'team1', 'team3'],
 'dev2': ['team2', 'team3', 'team1'],
 'dev3': ['team2', 'team3', 'team1'],
 'dev4': ['team1', 'team3', 'team2'],
 'dev5': ['team1', 'team3', 'team2'],
 'dev6': ['team3', 'team1', 'team2'],
 'dev7': ['team1', 'team3', 'team2'],
 'dev8': ['team3', 'team2', 'team1'],
 'dev9': ['team2', 'team3', 'team1']}
